In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

In [2]:
df_train = pd.read_csv("train_reshape.csv",sep = ",")
df_test = pd.read_csv("test_reshape.csv",sep = ",")

df_train.drop(["Parch","Fare"],axis = 1,inplace = True)
df_test.drop(["Parch","Fare"],axis = 1,inplace = True)

df_train["Dead"] = 0.0
df_train =  df_train.iloc[:,[0,5,1,2,3,4]]
df_train["Dead"][df_train["Survived"] == 0.0] =1.0

df_test["Survived"] = 0.0
df_test["Dead"] = 0.0
df_test =  df_test.iloc[:,[4,5,0,1,2,3]]

df_train = df_train.astype("float64")
df_test = df_test.astype("float64")

In [3]:
print(df_train.head())
print(df_test.head())

   Survived  Dead  Pclass  Sex   Age  SibSp
0       0.0   1.0     3.0  0.0  22.0    1.0
1       1.0   0.0     1.0  1.0  38.0    1.0
2       1.0   0.0     3.0  1.0  26.0    0.0
3       1.0   0.0     1.0  1.0  35.0    1.0
4       0.0   1.0     3.0  0.0  35.0    0.0
   Survived  Dead  Pclass  Sex   Age  SibSp
0       0.0   0.0     3.0  0.0  34.5    0.0
1       0.0   0.0     3.0  1.0  47.0    1.0
2       0.0   0.0     2.0  0.0  62.0    0.0
3       0.0   0.0     3.0  0.0  27.0    0.0
4       0.0   0.0     3.0  1.0  22.0    1.0


In [4]:
df_test.dtypes

Survived    float64
Dead        float64
Pclass      float64
Sex         float64
Age         float64
SibSp       float64
dtype: object

In [5]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_train)

MinMaxScaler(copy=True, feature_range=(-1, 1))

In [6]:
df_train_norm = scaler.transform(df_train)
df_test_norm = scaler.transform(df_test)

In [7]:
df_test_norm

array([[-1.        , -1.        ,  1.        , -1.        , -0.14350339,
        -1.        ],
       [-1.        , -1.        ,  1.        ,  1.        ,  0.17064589,
        -0.75      ],
       [-1.        , -1.        ,  0.        , -1.        ,  0.54762503,
        -1.        ],
       ...,
       [-1.        , -1.        ,  1.        , -1.        , -0.04297562,
        -1.        ],
       [-1.        , -1.        ,  1.        , -1.        , -0.24974641,
        -1.        ],
       [-1.        , -1.        ,  1.        , -1.        , -0.24974641,
        -0.75      ]])

In [8]:
X_train = df_train_norm[:,2:]
Y_train = df_train_norm[:,(0,1)]
X_test = df_test_norm[:,2:]
Y_test = df_test_norm[:,(0,1)]

In [9]:
Y_train

array([[-1.,  1.],
       [ 1., -1.],
       [ 1., -1.],
       ...,
       [-1.,  1.],
       [ 1., -1.],
       [-1.,  1.]])

### TensorFlow

In [10]:
net = tf.InteractiveSession()

# 初期化
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

#トレーニングデータの列数
n_stocks = X_train.shape[1]
 
# ニューロンの数を設定
n1 = 32
n2 = 65
n3 = 14
  
# プレースホルダーの作成（データ格納予定地的な）
X = tf.placeholder(dtype=tf.float32, shape=[None, n_stocks])
Y = tf.placeholder(dtype=tf.float32, shape=[None,2])
 

#重み、バイアスのオブジェクト(?)を生成

#W = tf.Variable(weight_initializer([n_stocks, 2]))
#b = tf.Variable(bias_initializer([2]))

W1 = tf.Variable(weight_initializer([n_stocks, n1]))
b1 = tf.Variable(bias_initializer([n1]))
h1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

W2 = tf.Variable(weight_initializer([n1,n2]))
b2 = tf.Variable(bias_initializer([n2]))
h2 = tf.nn.relu(tf.add(tf.matmul(h1, W2), b2))

W3 = tf.Variable(weight_initializer([n2,n3]))
b3 = tf.Variable(bias_initializer([n3]))
h3 = tf.nn.relu(tf.add(tf.matmul(h2, W3), b3))

W4 = tf.Variable(weight_initializer([n3,2]))
b4 = tf.Variable(bias_initializer([2]))

#出力
out = tf.nn.softmax(tf.matmul(h3,W4) + b4)

#コスト関数
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# 最適化関数
opt = tf.train.AdamOptimizer().minimize(mse)
    
# 初期化
net.run(tf.global_variables_initializer())

In [11]:
# ニューラルネットワークの設定
 
# 訓練開始！5000回の反復処理
epochs = 10000
for e in range(epochs):
    net.run(opt, feed_dict={X: X_train, Y: Y_train})
    loss_val = net.run(mse,feed_dict={X: X_train,Y:Y_train})
    #summary_writer.add_summary(loss_val,i)
    if e % 1000 == 0:
        print("Step: %d, Loss: %f" % (e,loss_val))
        
print("Step: %d, Loss: %f" % (epochs,loss_val))

Step: 0, Loss: 1.259393
Step: 1000, Loss: 0.789161
Step: 2000, Loss: 0.781413
Step: 3000, Loss: 0.777252
Step: 4000, Loss: 0.775489
Step: 5000, Loss: 0.774858
Step: 6000, Loss: 0.774686
Step: 7000, Loss: 0.774031
Step: 8000, Loss: 0.773851
Step: 9000, Loss: 0.773704
Step: 10000, Loss: 0.774606


In [12]:
pred_test = net.run(out, feed_dict={X: X_test})
pred_test[pred_test[:,0] < pred_test[:,1]] = [-1,1]
pred_test[pred_test[:,0] > pred_test[:,1]] = [1,-1]

pred_test = np.concatenate((pred_test, X_test), axis=1)

In [13]:
pred_inv = scaler.inverse_transform(pred_test)

ValueError: operands could not be broadcast together with shapes (418,4) (6,) (418,4) 

In [14]:
alive = pred_inv[:,0].astype(int)

NameError: name 'pred_inv' is not defined

In [15]:
alive

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [16]:
test = pd.read_csv("test.csv",sep = ",")

In [17]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
pid = np.array(test["PassengerId"]).astype(int)

In [19]:
solution = pd.DataFrame(alive,pid,columns = ["Survived"])

In [20]:
solution

,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,1
899,0
900,0
901,0


In [21]:
solution.to_csv("my_first_sol3.csv",index_label = ["PassengerId"])